In [133]:
import numpy as np
import sys
import os
import matplotlib.pyplot as plt
from nltk.tokenize import TweetTokenizer, word_tokenize, wordpunct_tokenize, sent_tokenize

In [134]:
with open('shakespeare.txt', 'r') as f:
    data = f.read().lower() #Reading in Sonnets in lowercase

In [135]:
sonnets = data.split('\n\n')
#Splitting each sonnet up based on double gap between poems
sonnets = [sonnet.strip() for sonnet in sonnets] #Removing whitespace
sonnets.pop(98) #Removing Sonnets with not 14 lines
sonnets.pop(124)
lines = [sonnet.split('\n') for sonnet in sonnets] #splitting up the sonnets into lines
lines = [line[1:] for line in lines] #Removing index of poem
quattrains = [line[0:4] for line in lines] + [line[4:8] for line in lines] + [line[8:12] for line in lines] #splitting data
couplets = [line[12:15] for line in lines]                                  #into quattrains and couplets

In [136]:
tknzr = TweetTokenizer()
#tokenizing each quattrain and coup line by line, not excluding bigrams/splitting hyphenated words
token_quat = [ [tknzr.tokenize(line) for line in quattrain] for quattrain in quattrains] 
token_coup = [ [tknzr.tokenize(line) for line in couplet] for couplet in couplets]


punct = set(['.', ',', '!', ':', ';'])

filtered_token_quat = [ [[word for word in line if word not in punct] for line in quattrain] for quattrain in token_quat]
concat_token_quat = [quattrain[0] + quattrain[1] + quattrain[2] + quattrain[3] for quattrain in filtered_token_quat]

filtered_token_coup = [ [[word for word in line if word not in punct] for line in couplet] for couplet in token_coup]
concat_token_coup = [couplet[0] + couplet[1] for couplet in filtered_token_coup]
